# Linear Regression 

Now we'll discuss a case study solution using multiple linear regression, and regularised linear regresison [Ridge and Lasso] . We'll also look at hyper parameter tuning for regularised regression.

In [3]:
import pandas as pd 
import numpy as np

A little background on the case study. This data belongs to a loan aggregator agency which connects loan applications to different financial institutions in attempt to get the best interest rate. They want to now utilise past data to predict interest rate given by any financial institute just by looking at loan application characteristics.

To achieve that , they have decided to do a POC with a data from a particular financial institution. The data is given in the file "loans data.csv". Lets begin: 

In [4]:
train_file='../data/loan_data_train.csv'
test_file='../data/loan_data_test.csv'

ld_train=pd.read_csv(train_file)
ld_test=pd.read_csv(test_file)               

In [5]:
print(ld_train.shape)
ld_train.head()

(2200, 15)


,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years


In [6]:
#test data does not have interest rate
print(ld_test.shape)
ld_test.head()

(300, 14)


,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20093,5000,5000,60 months,moving,12.59%,NY,RENT,4416.67,690-694,13,7686,0,< 1 year
1,62445,18000,18000,60 months,debt_consolidation,4.93%,CA,RENT,5258.50,710-714,6,11596,0,10+ years
2,65248,7200,7200,60 months,debt_consolidation,25.16%,LA,MORTGAGE,3750.00,750-754,13,7283,0,6 years
3,81822,7200,7200,36 months,debt_consolidation,17.27%,NY,MORTGAGE,3416.67,790-794,14,4838,0,10+ years
4,57923,22000,22000,60 months,debt_consolidation,18.28%,MI,MORTGAGE,6083.33,720-724,9,20181,0,8 years


In [8]:
# lets combine the data for data prep (cleaning)
ld_test['Interest.Rate']=np.nan
ld_train['data']='train'
ld_test['data']='test'
ld_test=ld_test[ld_train.columns] #to ensure same order of columns
ld_all=pd.concat([ld_train,ld_test],axis=0)

In [9]:
ld_all.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,data
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years,train
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years,train
2,67265.0,2100,2100,14.33%,36 months,major_purchase,3.50%,LA,OWN,1000.00,690-694,13,893,1.0,< 1 year,train
3,80167.0,28000,28000,16.29%,36 months,credit_card,19.62%,NV,MORTGAGE,7083.33,710-714,12,38194,1.0,10+ years,train
4,17240.0,24250,17431.82,12.23%,60 months,credit_card,23.79%,OH,MORTGAGE,5833.33,730-734,6,31061,2.0,10+ years,train


In [10]:
ld_all.dtypes

ID                                float64
Amount.Requested                   object
Amount.Funded.By.Investors         object
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
data                               object
dtype: object

In [ ]:
# ID,Amount.Funded.By.Investors : drop 
# Interest Rate , Debt to income ratio : remove % and then to numeric
# Amount.Requested , 'Open.CREDIT.Lines','Revolving.CREDIT.Balance': convert it to numeric 
# FICO.Range : replace it by a numeric column which is average of the range
# Employment Length : convert to number
# Loan Lenth, Loan Purpose , State , Home ownership: dummies for categories with good occurence rate

In [11]:
ld_all.drop(['ID','Amount.Funded.By.Investors'],axis=1,inplace=True)

variable `Interest.Rate` and `Debt.To.Income.Ratio` contain "%" sign in their values and because of which they have come as character columns in the data. Lets remove these percentages first.

In [12]:
for col in ['Interest.Rate','Debt.To.Income.Ratio']:
    ld_all[col]=ld_all[col].str.replace("%","")

We can see that many columns which should have really been numbers have been imported as character columns , probably because some characters values in those columns in the files. We'll convert all such columns to numbers .

In [13]:
for col in ['Amount.Requested', 'Interest.Rate','Debt.To.Income.Ratio',
            'Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
    ld_all[col]=pd.to_numeric(ld_all[col],errors='coerce') 

If we look at first few values of variable FICO.Range , we can see that we can convert it to numeric by taking average of the range given. To do that first we need to split the column with "-", so that we can have both end of ranges in separate columns and then we can simply average them.

In [14]:
k=ld_all['FICO.Range'].str.split("-",expand=True).astype(float)

ld_all['fico']=0.5*(k[0]+k[1])

del ld_all['FICO.Range']

In [15]:
ld_all['Employment.Length'].value_counts()

10+ years    653
< 1 year     249
2 years      243
3 years      235
5 years      202
4 years      191
1 year       177
6 years      163
7 years      127
8 years      108
9 years       72
.              2
Name: Employment.Length, dtype: int64

In [16]:
ld_all['Employment.Length']=ld_all['Employment.Length'].str.replace('years',"")
ld_all['Employment.Length']=ld_all['Employment.Length'].str.replace('year',"")

In [ ]:
#np.where(condition, value_if_True, value_if_False)
ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[:2]=="10",10,ld_all['Employment.Length'])
ld_all['Employment.Length']=np.where(ld_all['Employment.Length'].str[0]=="<",0,ld_all['Employment.Length'])

In [ ]:
ld_all['Employment.Length']=pd.to_numeric(ld_all['Employment.Length'],errors='coerce')

In [21]:
# Notice that to apply string function on pandas data frame columns you need to str attribute
cat_cols=ld_all.select_dtypes(['object']).columns

In [22]:
cat_cols

Index(['Loan.Length', 'Loan.Purpose', 'State', 'Home.Ownership', 'data'], dtype='object')

In [23]:
cat_cols=cat_cols[:-1]

In [24]:
cat_cols

Index(['Loan.Length', 'Loan.Purpose', 'State', 'Home.Ownership'], dtype='object')

In [25]:
# you can use following method if you want to ignore categories with too low frequencies ,
#in next section for logistic regression we will be using  pandas' get dummies function. 
# you can work with either of these . 
#ignoring categories with low frequencies however will result in fewer columns without 
# affecting model performance too much .

for col in cat_cols:
    freqs=ld_all[col].value_counts()
    k=freqs.index[freqs>20][:-1]
    for cat in k:
        name=col+'_'+cat
        ld_all[name]=(ld_all[col]==cat).astype(int)
    del ld_all[col]
    print(col)  

Loan.Length
Loan.Purpose
State
Home.Ownership


In [26]:
ld_all.shape

(2500, 51)

In [27]:
ld_all.isnull().sum()

Amount.Requested                     5
Interest.Rate                      300
Debt.To.Income.Ratio                 1
Monthly.Income                       3
Open.CREDIT.Lines                    9
Revolving.CREDIT.Balance             5
Inquiries.in.the.Last.6.Months       3
Employment.Length                   80
data                                 0
fico                                 0
Loan.Length_36 months                0
Loan.Purpose_debt_consolidation      0
Loan.Purpose_credit_card             0
Loan.Purpose_other                   0
Loan.Purpose_home_improvement        0
Loan.Purpose_major_purchase          0
Loan.Purpose_small_business          0
Loan.Purpose_car                     0
Loan.Purpose_wedding                 0
Loan.Purpose_medical                 0
Loan.Purpose_moving                  0
State_CA                             0
State_NY                             0
State_TX                             0
State_FL                             0
State_IL                 

In [28]:
#result = ld.loc[condition,desired_cols]
for col in ld_all.columns:
    if (col not in ['Interest.Rate','data'])& (ld_all[col].isnull().sum()>0):
        ld_all.loc[ld_all[col].isnull(),col]=ld_all.loc[ld_all['data']=='train',col].mean()

In [29]:
ld_all.isnull().sum()

Amount.Requested                     0
Interest.Rate                      300
Debt.To.Income.Ratio                 0
Monthly.Income                       0
Open.CREDIT.Lines                    0
Revolving.CREDIT.Balance             0
Inquiries.in.the.Last.6.Months       0
Employment.Length                    0
data                                 0
fico                                 0
Loan.Length_36 months                0
Loan.Purpose_debt_consolidation      0
Loan.Purpose_credit_card             0
Loan.Purpose_other                   0
Loan.Purpose_home_improvement        0
Loan.Purpose_major_purchase          0
Loan.Purpose_small_business          0
Loan.Purpose_car                     0
Loan.Purpose_wedding                 0
Loan.Purpose_medical                 0
Loan.Purpose_moving                  0
State_CA                             0
State_NY                             0
State_TX                             0
State_FL                             0
State_IL                 

In [30]:
ld_train=ld_all[ld_all['data']=='train']
del ld_train['data']
ld_test=ld_all[ld_all['data']=='test']
ld_test.drop(['Interest.Rate','data'],axis=1,inplace=True)

/anaconda3/envs/forall/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [31]:
del ld_all

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
ld_train1,ld_train2=train_test_split(ld_train,test_size=0.2,random_state=2)

In [34]:
# Notice that only train data is used for imputing missing values in both train and test 
x_train1=ld_train1.drop('Interest.Rate',axis=1)
y_train1=ld_train1['Interest.Rate']

In [35]:
from sklearn.linear_model import LinearRegression

In [36]:
lm=LinearRegression()

In [37]:
lm.fit(x_train1,y_train1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [38]:
x_train1.shape

(1760, 49)

In [39]:
lm.intercept_

75.92121413594562

In [40]:
list(zip(x_train1.columns,lm.coef_))

[('Amount.Requested', 0.00015602405153876978),
 ('Debt.To.Income.Ratio', -0.003938504173760232),
 ('Monthly.Income', -2.6568573569333197e-05),
 ('Open.CREDIT.Lines', -0.03992260834024737),
 ('Revolving.CREDIT.Balance', -3.9236478586906535e-06),
 ('Inquiries.in.the.Last.6.Months', 0.3361172111313327),
 ('Employment.Length', 0.034993671894509346),
 ('fico', -0.08667701121950722),
 ('Loan.Length_36 months', -3.1437472469505927),
 ('Loan.Purpose_debt_consolidation', -0.46739356903558904),
 ('Loan.Purpose_credit_card', -0.6069873604061786),
 ('Loan.Purpose_other', 0.44417142270192655),
 ('Loan.Purpose_home_improvement', -0.36118998491814186),
 ('Loan.Purpose_major_purchase', -0.09589524932665905),
 ('Loan.Purpose_small_business', 0.06800548772823696),
 ('Loan.Purpose_car', 0.02525962803601135),
 ('Loan.Purpose_wedding', -0.7791542650056439),
 ('Loan.Purpose_medical', -0.428115210995497),
 ('Loan.Purpose_moving', 1.2845276544595674),
 ('State_CA', -0.21159715256759146),
 ('State_NY', -0.1277

In [ ]:
x_train2=ld_train2.drop('Interest.Rate',axis=1)

In [ ]:
predicted_ir=lm.predict(x_train2)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(ld_train2['Interest.Rate'],predicted_ir)

We know the tentative performance now, lets build the model on entire training to make prediction on test/production

In [ ]:
x_train=ld_train.drop('Interest.Rate',axis=1)
y_train=ld_train['Interest.Rate']

In [ ]:
lm.fit(x_train,y_train)

In [ ]:
test_pred=lm.predict(ld_test)

We can write these to a csv file for submission like this :

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

# Ridge  Regression

In [ ]:
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV

In [ ]:
lambdas=np.linspace(1,100,100)

In [ ]:
params={'alpha':lambdas}

In [ ]:
model=Ridge(fit_intercept=True)

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.cv_results_

 if you want you can now fit a ridge regression model with obtained value of alpha , although there is no need, grid search automatically fits the best estimator on the entire data, you can directly use this to make predictions on test_data. But if you want to look at coefficients , its much more convenient to fit the model with direct function

Using the report function given below you can see the cv performance of top few models as well, that will the tentative performance

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(grid_search.cv_results_,100)

In [ ]:
test_pred=grid_search.predict(ld_test)

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

## For looking at coefficients

In [ ]:
ridge_model=grid_search.best_estimator_

In [ ]:
ridge_model.fit(x_train,y_train)

In [ ]:
list(zip(x_train1.columns,ridge_model.coef_))

## Lasso Regression

In [ ]:
lambdas=np.linspace(1,10,100)
model=Lasso(fit_intercept=True)
params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

you can see that, the best value of alpha comes at the edge of the range that we tried , we should expand the trial range on that side and run this again

In [ ]:
lambdas=np.linspace(.001,2,100)
params={'alpha':lambdas}

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
lasso_model=grid_search.best_estimator_

In [ ]:
lasso_model.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,lasso_model.coef_))

# Logistic Regression

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_file='../data/rg_train.csv'
test_file='../data/rg_test.csv'
bd_train=pd.read_csv(train_file)

bd_test=pd.read_csv(test_file)


bd_test['Revenue.Grid']=np.nan
bd_train['data']='train'
bd_test['data']='test'
bd_test=bd_test[bd_train.columns]
bd_all=pd.concat([bd_train,bd_test],axis=0)

In [ ]:
bd_all['Revenue.Grid'].value_counts()

In [ ]:
list(zip(bd_all.columns,bd_all.dtypes,bd_all.nunique()))

In [ ]:
# REF_NO,post_code , post_area  : drop 
# children : Zero : 0 , 4+ : 4 and then convert to numeric
# age_band : dummies 
# status , occupation , occupation_partner , home_status,family_income : dummies
# self_employed, ` : dummies
# TVArea , Region , gender : dummies
# Revenue Grid : 1,2 : 1,0

In [ ]:
bd_all.drop(['REF_NO','post_code','post_area'],axis=1,inplace=True)

In [ ]:
bd_all['children']=np.where(bd_all['children']=='Zero',0,bd_all['children'])
bd_all['children']=np.where(bd_all['children'][:1]=='4',4,bd_all['children'])
bd_all['children']=pd.to_numeric(bd_all['children'],errors='coerce')

In [ ]:
bd_all['Revenue.Grid']=(bd_all['Revenue.Grid']==1).astype(int)

In [ ]:
cat_vars=bd_all.select_dtypes(['object']).columns

cat_vars

In [ ]:
for col in cat_vars[:-1]:
    dummy=pd.get_dummies(bd_all[col],drop_first=True,prefix=col)
    bd_all=pd.concat([bd_all,dummy],axis=1)
    del bd_all[col]
    print(col)
del dummy

In [ ]:
bd_all.shape

In [ ]:
bd_all.isnull().sum()

In [ ]:
bd_all.loc[bd_all['children'].isnull(),'children']=bd_all.loc[bd_all['data']=='train','children'].mean()


In [ ]:
bd_train=bd_all[bd_all['data']=='train']
del bd_train['data']
bd_test=bd_all[bd_all['data']=='test']
bd_test.drop(['Revenue.Grid','data'],axis=1,inplace=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.01,1000,10)}

In [ ]:
model=LogisticRegression(fit_intercept=True)

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=5,scoring="roc_auc")

In [ ]:
x_train=bd_train.drop('Revenue.Grid',axis=1)
y_train=bd_train['Revenue.Grid']

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
logr=grid_search.best_estimator_

In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
logr.fit(x_train,y_train)

In [ ]:
cutoffs=np.linspace(0.01,0.99,99)

cutoffs

In [ ]:
train_score=logr.predict_proba(x_train)[:,1]

real=y_train

In [ ]:
train_score>0.2

In [ ]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)

    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()
    
    P=TP+FN
    N=TN+FP
      
    KS=(TP/P)-(FP/N)
    
    
    KS_all.append(KS)

# try out what cutoffs you get when you use F_beta scores with different values of betas [0.5 , 5]
# beta < 1 : you will get cutoff , which is high ( favours precision)
# beta > 1 : you will get cutoff , which is low (favours precision )

In [ ]:
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff

In [ ]:
logr.intercept_

In [ ]:
list(zip(x_train.columns,logr.coef_[0]))

if you simply had to submit probability scores , you could do this 

In [ ]:
test_score=logr.predict_proba(bd_test)[:,1]
pd.DataFrame(test_score).to_csv("mysubmission.csv",index=False)

if you had to submit hardclasses , you can apply the cutoff obtained above and then submit

In [ ]:
test_classes=(test_score>mycutoff).astype(int)

In [ ]:
pd.DataFrame(test_classes).to_csv("mysubmission.csv",index=False)